In [67]:
import networkx as nx
import time 
import pandas as pd
from utils.distance import get_distance
import matplotlib.pyplot as plt
import numpy as np
from utils.nn import nearest_neighbor
from utils.genalg import genetic_alg
from utils.simann import simulated_annealing
from utils.aco import ant_colony
from data import locations_list


df = pd.read_csv("calculations/road_distances.csv")

Graph = nx.Graph()
for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
    
results = []
ITERATIONS = range(10)

In [68]:
Graph = nx.Graph()
for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
# Nearest Neighbor
print("Running Nearest Neighbor")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = nearest_neighbor(Graph, start_index=list(Graph.nodes).index('Bulacan Provincial Capitol Malolos'))
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
nn_time = time.time() - start
print(f"NN Finished: {nn_time:.5f}s | Distance: {best_distance:.5f}m")

results.append(("Nearest Neighbor", nn_time, best_distance, best_tour))

Running Nearest Neighbor
NN Finished: 0.00088s | Distance: 126370.67504m


In [69]:
# Genetic Algorithm
Graph = nx.Graph()
for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
print("Running Genetic Algorithm")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = genetic_alg(Graph, 100, 0.01, 1000)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
ga_time = time.time() - start
print(f"GA Finished: {ga_time} | Distance: {best_distance}")
results.append(("Genetic Algorithm", ga_time, best_distance, best_tour))



Running Genetic Algorithm
GA Finished: 16.475030183792114 | Distance: 110712.44215334037


In [70]:

# Simulated Annealing
Graph = nx.Graph()
for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
print("Running Simulated Annealing")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = simulated_annealing(Graph, 10_000, 0.995,20_000)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
sa_time = time.time() - start
print(f"SA Finished: {sa_time:.5f}s | Distance: {best_distance:.5f}m")
results.append(("Simulated Annealing", sa_time, best_distance, best_tour))


Running Simulated Annealing
SA Finished: 0.58222s | Distance: 110397.17670m


In [71]:
df[df["Loc1"] == "Alberta Uitangcoy Santos House Malolos"]

,Unnamed: 0,Loc1,Loc2,Distances(m)


In [72]:
Graph = nx.Graph()
for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
import time
ANTS, EPOCHS, ALPHA, BETA, RHO, Q = 50, 200, 1.0, 5.0, 0.1, 100

print("Running Ant Colony Optimization")
start = time.time()
best_tour, best_distance = None, float('Inf')
for _ in ITERATIONS:
    tour, distance = ant_colony(Graph, ANTS, EPOCHS, ALPHA, BETA, RHO, Q)
    if distance < best_distance:
        best_tour = tour
        best_distance = distance
ac_time = time.time() - start
print(f"SA Finished: {ac_time:.5f}s | Distance: {best_distance:.5f}m")
results.append(("Ant Colony Optimization", ac_time, best_distance, best_tour))

Running Ant Colony Optimization
SA Finished: 42.88879s | Distance: 110596.73380m


In [73]:
res_df = pd.DataFrame(results, columns=["Algorithm", "Time (s)", "Distance", "Tour"]).set_index("Algorithm")

In [ ]:
res_df.to_csv("calculations/solution_results.csv")

In [74]:
res_df

,Time (s),Distance,Tour
Algorithm,,,
Nearest Neighbor,0.000883,126370.675040,"(Bulacan Provincial Capitol Malolos, Hiyas Bul..."
Genetic Algorithm,16.475030,110712.442153,"(Municipal Trial Court Pulilan, Parish of St J..."
Simulated Annealing,0.582220,110397.176703,"(Municipal Trial Court Pulilan, St Augustine P..."
Ant Colony Optimization,42.888794,110596.733799,"(St Francis of Aggigi Parish Meycauayan, Old M..."


In [75]:
remap = {
"St John of God Parish San Rafael": 'C0',
"Old Municipal Building Baliwag": 'C1',
"St Augustine Parish Baliwag": 'C2',
"St James the Apostle Parish Plaridel": 'C3',
"Simboryo Chapel of Quingua Plaridel": 'C4',
"Municipal Trial Court Pulilan": 'C5',
"Parish of St John the Baptist Calumpit": 'C6',
"Barasoain Church Malolos": 'C7',
"Immaculate Conception Parish Malolos": 'C8',
"Museum of Philippine Political History Malolos": 'C9',
"Malolos of Aguas Potables Malolos": 'C10',
"Hiyas Bulacan Cultural Center and Museum Malolos": 'C11',
"Bulacan Provincial Capitol Malolos": 'C12',
"Alberta Uitangcoy Santos House Malolos": 'C13',
"Dr Luis Santos House": 'C14',
"Old PNR  Guiguinto Station Guiguinto": 'C15',
"Constantino Ancestral HouseBalagtas": 'C16',
"Diocsan Shrine Bulakan": 'C17',
"Old Meycauayan PNR Station Meycauayan": 'C18',
"St Francis of Aggigi Parish Meycauayan": 'C19'
}

for i in [remap[i] for i in res_df.iloc[0]["Tour"]]:
    print(i,end=", ")

C12, C11, C7, C9, C8, C14, C10, C3, C4, C5, C6, C15, C16, C17, C18, C19, C2, C1, C0, 

In [76]:
from utils.location_graph import LocationGraph
locGraph = LocationGraph("Bulacan, Philippines")

In [ ]:
import osmnx as ox
import plotly.graph_objects as go
import plotly.express as px


# Get the street network between the two points
# Iterate through the results dataframe
for algo, time, distance, tour in res_df.itertuples():
    print("Handling ", algo)
    # Extract coordinates for the path.  Handles empty tours.
    fig = go.Figure() # create figure outside the loop

    for i in range(len(tour)):
        loc1 = tour[i]
        loc2 = tour[(i + 1) % len(tour)]

        loc1 = next((l for l in locations_list if l.name == loc1))
        loc2 = next((l for l in locations_list if l.name == loc2))

        route, dist = locGraph.get_path(loc1, loc2)
        path_coords = [(locGraph.G.nodes[node]['y'], locGraph.G.nodes[node]['x']) for node in route]
        fig.add_trace(go.Scattermap(
            mode="lines",
            lon=[b for a, b in path_coords],
            lat=[a for a, b in path_coords],
            marker={'size': 10},
            line={'color': 'blue'},
            name=""
        ))

    node_long = [loc.long for loc in locations_list] # corrected order
    node_lat = [loc.lat for loc in locations_list]

    # Add the start points.  Handles empty locations_list.
    if locations_list:
        fig.add_trace(go.Scattermap(
            mode="markers+text",
            lon=node_long,
            lat=node_lat,
            marker={'size': 15, 'color': 'green'},
            name='Locations',  # Changed name to plural, more accurate,
            text=[remap[loc.name] for loc in locations_list]
        ))

    # Update the layout to include mapbox access token and style
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=10,
        margin={'r': 0, 't': 0, 'b': 0, 'l': 0}
    )
    fig.show()

Handling  Nearest Neighbor


Handling  Genetic Algorithm


Handling  Simulated Annealing
